# <center> Polytech Paris Saclay | 3ème année


## <center> TP2 Calcul scientifique: la factorisation LU

### Rappel

La factorisation LU consiste à factoriser une matrice A en un produit de deux matrices: une matrice triangulaire inférieure L et une matrice triangulaire supérieure U telles que A = L\*U. Le système Ax=b devient alors LUx=b. Une telle factorisation permet de résoudre plus facilement le nouveau système: deux résolutions successives: une par descente et une par remontée.

       Algorithm 1: In place LU factorisation without pivoting
       Require: A is a n*n matrix
        1: for k <- 1 to n-1 do
        2:    for i <- k+1 to n do
        3:       A(i,k) <- A(i,k)/A(k,k)
        4:    end for
        5:    for i <- k+1 to n do
        6:       for j <- k+1 to n do
        7:          A(i,j) <- A(i,j) - A(i,k)*A(k,j)
        8:       end for
        9:    end for
        10: end for
   

- écrire une fonction *monLU(A::Matrix{Float64})* qui met en oeuvre la factorisation LU sans pivotage.

- On veut résoudre *LUx = b*. On pose *Ux = z* et on résout d'abord *Lz = b*. Il faut donc écrire une fonction *descente(L::Matrix{Float64}, b::Vector{Float64})* qui retourne *z* en mettant en oeuvre la résolution par decente d'un système triangulaire inférieur.

- On veut maintenant résoudre *Ux = z*. Il faut donc écrire une fonction *remontee(U::Matrix{Float64}, z::Vector{Float64})* qui retourne *x* en mettant en oeuvre la résolution par remontée d'un système triangulaire supérieur.

- écrire une fonction *monPivotLu(A::Matrix{Float64})* qui met en oeuvre la factorisation LU avec pivotage partiel.

On rappelle que le pivotage partiel consiste à échanger *le cas échéant* 2 lignes dans la matrice A (à partir de la ligne concernée par le pivot) de façon à placer en position de pivot l'élément de la colonne qui a la plus grande valeur absolue. En effet, ce pivot étant utilisé comme diviseur dans la suite de l'algorithme, il vaut mieux le choisir le plus grand possible pour des raisons de stabilité numérique.

- tester vos fonctions en résolvant les systèmes d'équations suivants:
  1. premier système à résoudre:
$$\begin{align*} 
x_1 + x_2 + 2x_3 &=  8 \\ 
-x_1 - 2x_2 + 3x_3 &=  1 \\
3x_1 - 7x_2 + 4x_3 &= 10
\end{align*}$$
  1. deuxième système à résoudre:
$$\begin{align*} 
x_1 - x_2 + 2x_3 - x_4 &= -1 \\ 
2x_1 + x_2 - 2x_3 - 2x_4 &= -2 \\
-x_1 + 2x_2 - 4x_3 + x_4 &=  1 \\
3x_1 - 3x_4 &= -3
\end{align*}$$

# <font color="purple"> Solution

### Sans pivot

La première question consistait à traduire le pseudo code donné en un code Julia opérationnel. C'est un exercice pas si trivial qu'il en a l'air car il faut faire abstraction de la syntaxe du pseudo code pour écrire du vrai code Julia et surtout il faut penser à tous les à-côtés (récupérer les bonnes variables, retourner les bonnes matrices etc.).  

Les fonctions qu'il fallait utiliser étaient les suivantes : <br>
- (n,m) = size(A) qui retourne respectivement le nombre de lignes et de colonnes de A. Similairement on pouvait utiliser n = size(A,1) ou m = size(A,2). 
- L = tril(A) et U = triu(A) retournent les parties triangulaires inférieure et supérieure de A. tril(A,-1) et triu(A-1) permettent de récupérer les parties triangulaires sans les diagonales.

Après exécution du pseudo code "Algorithm 1" on modifie directement la matrice $A$ en entrée et on se retrouve avec 
$$ A = \begin{pmatrix}   *                                    \\
                         & *             &   & \huge{U}     \\
                         &               & *                \\
                         & \huge{L}      &    & *            \\
                         &               &    &   & * \end{pmatrix} $$ 
                         
Un point important est que la factorisation $LU$ n'est pas unique : pour n'importe quelle matrice diagonale $D$ inversible on a 
$$ A = LDD^{-1}U = L'U'$$ 
où $L', U'$ sont de nouvelles matrices triangulaires valides. Par convention on choisit donc $L$ et $U$ de sorte que la diagonale de $L$ ne contient que des 1. Au-delà de l'avantage de fixer une convention, cela permet surtout de stocker $L$ et $U$ entièrement dans $A$ car on n'a pas besoin de stocker la diagonale de $L$. Si $L$ et $U$ avaient deux diagonales spécifiques on peut se convaincre qu'il n'est pas possible de stocker ces deux diagonales dans $A$.

On obtient finalement le code suivant : 

In [5]:
using LinearAlgebra

In [6]:
function monLU(A)
   
    n,m = size(A)
    @assert n==m        # pour vérifier que A est carrée
    
    for k=1:n-1
        for i=k+1:n
            @assert A[k,k] != 0    # on ne doit pas diviser par 0
            A[i,k] = A[i,k]/A[k,k]
        end
        for i=k+1:n
            for j=k+1:n
                A[i,j] = A[i,j] - A[i,k]*A[k,j]
            end
        end
    end
    
    L = tril(A,-1) + Matrix{eltype(A)}(I, n, n)
    U = triu(A)
    
    return L,U    
end

monLU (generic function with 1 method)

On vérifie que notre fonction marche bien sur des cas gentils : 

In [12]:
average_error = 0
n = 100
niter = 100
for i=1:niter
    A = rand(n,n);
    L,U = monLU(copy(A)) # la copie est nécessaire pour vérifier nos résultats
    average_error += norm(A - L*U)
end

println("Erreur moyenne de monLU : ", average_error/niter)

Erreur moyenne de monLU : 4.17354476811029e-12


Une fois que l'on a $A = LU$, on peut résoudre $Ax = b$ en résolvant $LUx = b$. Pour cela on va résoudre 
$$ Lz = b $$ 
en premier avec $z = Ux$. On calcule une solution $z_0$ et on résout alors $$Ux = z_0$$ ce qui nous donnera la solution au système initial $Ax = b$.

La résolution d'un système triangulaire est plus simple à mettre en oeuvre. On propose une solution récursive. Supposons que $n = 5$ pour illustrer l'algorithme. Si on écrit 
$$ L = \begin{pmatrix} l_{11} \\ 
                       l_{21} & l_{22} \\
                       l_{31} & l_{32} & l_{33} \\
                       l_{41} & l_{42} & l_{43} & l_{44} \\
                       l_{51} & l_{52} & l_{53} & l_{54} & l_{55} \end{pmatrix} $$
on pose 
$$ L' = \begin{pmatrix}l_{22} \\
                       l_{32} & l_{33} \\
                       l_{42} & l_{43} & l_{44} \\
                       l_{52} & l_{53} & l_{54} & l_{55} \end{pmatrix} $$
et on réécrit le système $Lz = b$ de la manière suivante :  
$$ Lz = b \;\;\; \equiv \;\;\; z_1 \begin{pmatrix} l_{11} \\ l_{21} \\ l_{31} \\ l_{41} \\ l_{51} \end{pmatrix} + \begin{pmatrix} 0 \\ L' \begin{pmatrix} z_2 \\ z_3 \\ z_4 \\ z_5 \end{pmatrix} \end{pmatrix} = \begin{pmatrix} b_1 \\ b_2 \\ b_3 \\ b_4 \\ b_5 \end{pmatrix}. $$ 

La première composante de notre système d'équations nous donne 
$$ z_1 l_{11} = b_1 $$ i.e $$ z_1 = \frac{b_1}{l_{11}}. $$

Ensuite il nous reste à résoudre 
$$ L' \begin{pmatrix} z_2 \\ z_3 \\ z_4 \\ z_5 \end{pmatrix} = \begin{pmatrix} b_2 \\ b_3 \\ b_4 \\ b_5 \end{pmatrix} - \frac{b_1}{l_{11}} \begin{pmatrix} l_{21} \\ l_{31} \\ l_{41} \\ l_{51} \end{pmatrix}  \; \; \; \equiv \; \; \; L'z'= b'. $$ 

où $L'$ est encore triangulaire inférieure mais de taille $n-1$, de même que $z'$ et $b'$ sont de taille $n-1$. On peut donc appeler notre fonction descente récursivement sur un problème de plus petite taille. Quand finalement la taille de notre système est à 1 on aura une seule équation à résoudre et on pourra retourner directement la solution sans avoir besoin de rappeler notre fonction.

On propose donc le code suivant pour descente : 

In [22]:
function descente(L, b)
    
    n,m = size(L)
    @assert n == m == length(b)  # on teste que les dimensions sont bonnes 
    
    z = Vector(undef, n)         # on initialise notre vecteur solution  
    z[1] = b[1]/L[1,1]           # on calcule la première composante de la solution 
    
    if (n==1)                    # si la taille du problème est 1 on s'arrête là 
        return z   
    else                         # sinon on appelle récursivement notre fonction sur un problème plus petit
        z[2:n] = descente(L[2:n, 2:n], b[2:n] - z[1].*L[2:n,1])
        return z
    end
end

descente (generic function with 1 method)

On teste notre fonction descente : 

In [54]:
average_error = 0
n = 100
niter = 100
for i=1:niter
    A = rand(n,n);
    L = tril(A) + Matrix{Float64}(I, n, n)   # on ajoute l'identité pour garantir la stabilité de la solution, testez sans et vous verrez l'erreur explose
    b = rand(n)
    z = descente(L,b)
    average_error += norm(L*z - b)
end

println("Erreur moyenne de descente : ", average_error/niter)

Erreur moyenne de descente : 4.425570217147702e-15


On peut coder de la même manière la remontée ou bien réutiliser le code de la descente. En effet un système triangulaire supérieur n'est qu'un système triangulaire inférieur où l'ordre des variables a été inversé : 
$$ U = \begin{pmatrix} u_{11} & u_{12} & u_{13} & u_{14} & u_{15} \\
                       & u_{22} & u_{23} & u_{24} & u_{25} \\
                       & & u_{33} & u_{34} & u_{35} \\
                       & & & u_{44} & u_{45} \\
                       & & & & u_{55}  
        \end{pmatrix} \begin{pmatrix} x_1 \\ x_2 \\ x_3 \\x_4 \\ x_5 \end{pmatrix} = \begin{pmatrix} z_1 \\ z_2 \\ z_3 \\ z_4 \\ z_5 \end{pmatrix} \; \; \; \equiv  \; \; \; L = \begin{pmatrix}  u_{55} \\ 
                       u_{45} & u_{44} \\
                       u_{35} & u_{34} & u_{33} \\
                       u_{25} & u_{24} & u_{23} & u_{22} \\
                       u_{15} & u_{15} & u_{13} & u_{12} & u_{11} \end{pmatrix} \begin{pmatrix} x_5 \\ x_4 \\ x_3 \\x_2 \\ x_1 \end{pmatrix} = \begin{pmatrix} z_5 \\ z_4 \\ z_3 \\ z_2 \\ z_1 \end{pmatrix}$$


Plutôt que de réécrire un code pour la remontée, on va appeler notre code descente. Pour inverser l'ordre des variables on va devoir inverser les lignes et colonnes de nos matrices et vecteurs. Il y a deux manières de faire : <br>
- appeler la fonction reverse. Sur des vecteurs cela inverse automatiquement, sur une matrice il faut préciser la dimension avec le mot clé "dims" et préciser 1 ou 2 (1 pour les lignes, 2 pour les colonnes). Dans notre cas on veut inverser et les lignes et les colonnes.
- manipuler directement les indices : end permet de récupérer la dernière composante d'un vecteur ou d'une ligne/colonne d'une matrice. En faisant vecteur[end:-1:1] on demande d'extraire dans vecteur les indices de end à 1 avec un incrément de -1, donc si end = n on récupère vecteur[n, n-1, n-2, n-3..., 1] et on a bien inversé l'ordre des éléments du vecteur.

In [65]:
function remontee(U, z)
    solution = descente(U[end:-1:1, end:-1:1], z[end:-1:1])
    return solution[end:-1:1]
end

remontee (generic function with 1 method)

In [66]:
average_error = 0
n = 100
niter = 100
for i=1:niter
    A = rand(n,n);
    U = triu(A) + Matrix{Float64}(I, n, n)   # on ajoute l'identité pour garantir la stabilité de la solution, testez sans et vous verrez l'erreur explose
    b = rand(n)
    z = remontee(U,b)
    average_error += norm(U*z - b)
end

println("Erreur moyenne de remontee : ", average_error/niter)

Erreur moyenne de remontee : 3.781405332964793e-15


On peut alors combiner nos trois fonctions (LU, Descente, Remontée) pour résoudre un système linéaire quelconque : 

In [67]:
function solve(A, b)
    
    L,U = monLU(copy(A))
    z = descente(L, b)
    x = remontee(U, z)
    
    return x
end

solve (generic function with 1 method)

In [72]:
average_error = 0
n = 100
niter = 100
for i=1:niter
    A = rand(n,n);
    b = rand(n)
    x = solve(A,b)
    average_error += norm(A*x - b)
end

println("Erreur moyenne de solve : ", average_error/niter)

Erreur moyenne de solve : 1.1416772114850254e-11


### Avec pivot

Avec pivot le code reste très similaire au code sans pivot avec deux difficultés principales : <br>
- il faut détecter à chaque colonne l'élément de plus forte amplitude et effectuer le pivot,
- il faut gérer astucieusement l'accumulation des échanges de lignes. 

Pour détecter l'élément de plus forte amplitude à chaque colonne il suffit de parcourir la colonne en question et de maintenir deux variables : <br>
- une variable **a_max** qui contient l'élément de plus forte amplitude rencontré jusque là,
- une variable **i_max** qui nous donne la ligne à laquelle cet élément a été rencontré.

Si en parcourant la colonne on tombe sur un élément de plus forte amplitude que **a_max** cela veut dire que c'est l'élément de plus forte amplitude rencontré jusque là, on met alors à jour **a_max** avec ce nouvel élément et on met à jour **i_max** également.

Après exécution de notre algorithme on doit avoir trois matrices : L, U et une matrice de permutation P telles que 
$$ PA = LU. $$ P se forme en accumulant progressivement les échanges entre les deux lignes, toutefois il peut être laborieux de manipuler P. L'avantage d'un langage comme Julia c'est qu'on peut traiter toute permutation comme un vecteur d'indices. En effet au lieu de faire $PA$ on peut faire $A[Q,:]$ où $Q$ est une permutation de $[|\,1,n\, |]$. De la même manière faire $AP$ est équivalent à $A[:,Q]$ (on échange les colonnes et non plus les lignes). 

$Q$ donne le nouvel ordre d'indiçage des lignes de $A$, c'est-à-dire que $Q[j] = $ le numéro de la ligne de $A$ qui est à la j-ième ligne de $PA$. 

Illustration sur une matrice $4\times4$ : 
$$ A = \begin{pmatrix} 4 & 7 & 5 & 5 \\ 3 & 4 & 3 & 5 \\ 8 & 6 & 10 & 2 \\ 3 & 7 & 1 & 9 \end{pmatrix} $$
et on veut appliquer la matrice de permutation 
$$ P = \begin{pmatrix} 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \\ 1 & 0 & 0 & 0 \end{pmatrix}. $$
On veut 
$$ PA = \begin{pmatrix} 3 & 4 & 3 & 5 \\ 3 & 7 & 1 & 9 \\ 8 & 6 & 10 & 2 \\ 4 & 7 & 5 & 5 \end{pmatrix} $$
ce qui revient à faire $A[ [2, 4, 3, 1], :]$ et $Q = [2,4,3,1]$. Ici par exemple on a bien $Q[1] = 2$ et donc la première ligne de $PA$ est bien la 2e ligne de $A$. De manière générale le vecteur d'indices $Q$ se calcule à partir de $P$ par la formule 
$$ \forall j \in [1,n], \; \; Q[j] = k \text{ de sorte que } P[j,k] = 1 $$
mais dans notre cas la manipulation de $Q$ est plus immédiate. 

Pendant l'exécution de l'algorithme on manipule une matrice $PA$ ($P$ étant associée à un vecteur d'indices $Q$) et $P$, donc $Q$, sont amenés à être modifiés pendant l'algorithme au fur et à mesure que l'on échange des lignes. Comment caractériser la modification sur $Q$ d'un échange de deux lignes ? La réponse est la suivante : si je veux échanger la ligne $j$ avec la ligne $k$ de $PA$, sachant que $Q[j] = i_1$ et $Q[k] = i_2$ cela veut dire qu'avant l'échange la $j$-ème ligne de $PA$ était la $i_1$-ème ligne de $A$ et la $k$-ème ligne de $PA$ était la $i_2$-ème ligne de $PA$. Après échange la $j$-ème ligne de $PA$ ($P$ a été updatée par l'échange des lignes) devient la $i_2$-ème ligne de $A$ et la $k$-ème ligne de $PA$ devient la $i_1$-ème ligne de $PA$ : il suffit donc de faire $Q[j] = i_2$ et $Q[k] = i_1$ c'est-à-dire échanger les valeurs de $Q$ aux emplacements $j$ et $k$.

On a tous les éléments pour implémenter notre LU avec pivot :

In [91]:
function monLUPivot(A)
    n,m = size(A)
    @assert n==m
    
    # On initialise Q à l'identité
    Q = collect(1:n)
    
    # Pour chaque ligne 
    for k=1:n-1
        # On initialise notre a_max à -Inf et i_max à 0
        a_max = -Inf
        i_max = 0
        # On parcourt la colonne mais uniquement les éléments sous diagonaux car la partie sur diagonale a déjà été traitée
        for j=k+1:n
            # Si l'amplitude de l'élément A[j,k] est plus grand que a_max alors c'est l'élément de plus forte amplitude parmi tous les A[i,k] avec i <= j.
            if abs(A[j,k]) > a_max
                a_max = abs(A[j,k])
                i_max = j
            end
        end 
        
        # Raccourci Julia pour échanger les valeurs de deux variables : a,b = b,a 
        # Ce raccourci marche aussi pour des éléments d'un vecteur ou d'une matrice
        A[k,:], A[i_max,:] = A[i_max,:], A[k,:]
        Q[i_max], Q[k] = Q[k], Q[i_max]
   
        # Partie classique du LU sans pivot
        if A[k,k] != 0
            for i=k+1:n
                A[i,k] = A[i,k]/A[k,k]
            end

            for i=k+1:n
                for j=k+1:n
                    A[i,j] = A[i,j] - A[i,k]*A[k,j]
                end
            end
        else
            println("Matrice non inversible !")
            @assert A[k,k] != 0
        end
    end
    
    L = tril(A,-1) + Matrix{eltype(A)}(I, n, n)
    U = triu(A)
    
    return L,U,Q
end

monLUPivot (generic function with 1 method)

On peut alors tester notre fonction sur des exemples aléatoires comme précédemment. Ici on doit avoir $A[Q,:] = LU$.

In [93]:
average_error = 0
n = 100
niter = 100
for i=1:niter
    A = rand(n,n);
    L,U,Q = monLUPivot(copy(A)) # la copie est nécessaire pour vérifier nos résultats
    average_error += norm(A[Q,:] - L*U)
end

println("Erreur moyenne de monLUPivot : ", average_error/niter)

Erreur moyenne de monLUPivot : 2.7637500422596913e-14


Pour résoudre un système linéaire, il faut s'apercevoir que l'on a maintenant 
$$ PA = LU = Pb $$ 
c'est-à-dire en terme de réindiçage 
$$ A[Q,:] = LU = b[Q].$$

On doit donc résoudre la descente sur b[Q] 
$$ Lz = b[Q] $$ 
et on aura une solution $z_0$ de sorte que $Lz_0 = b[Q]$. On veut résoudre ensuite 
$$ Ux = z_0 $$
ce qui nous donne una solution $x_0$. Cette solution est telle que 
$$ PAx_0 = Pb $$ 
et multiplier par $P^{-1}$ donnera 
$$ Ax_0 = b$$
donc $x_0$ reste la solution à notre problème, il n'y a pas besoin de réappliquer un réindiçage dessus. 

Cela nous donne la fonction de résolution : 

In [94]:
function solvePivot(A, b)
    
    L,U,Q = monLUPivot(copy(A))
    z = descente(L, b[Q])
    x = remontee(U, z)
    
    return x
end

solvePivot (generic function with 1 method)

On teste sur nos deux exemples : 

In [95]:
# Test 1
A = Matrix{Float64}([1 1 2 ; -1 -2 3 ; 3 -7 4])
y = [8,1,10]

# Sans pivot 
x = solve(A,y)
norm(A*x-y)

0.0

In [96]:
# Avec pivot 
x = solvePivot(A,y)
norm(A*x-y)

8.881784197001252e-16

In [97]:
# Test 2
A = Matrix{Float64}([1 -1 2 -1 ; 2 1 -2 -2 ; -1 2 -4 1 ; 3 0 0 -3])
y = [-1, -2, 1, -3]

# Sans pivot
x = solve(A,y)
norm(A*x-y)

AssertionError: AssertionError: A[k, k] != 0

In [98]:
# Avec pivot
x = solvePivot(A,y)
norm(A*x-y)

Matrice non inversible !


AssertionError: AssertionError: A[k, k] != 0

In [99]:
det(A)

0.0

Si la matrice n'est pas inversible, la méthode via LU (avec ou sans pivot) ne peut pas retourner de résultat ! 